# **Feature Engineering Walkthrough**

#### **Description**: In this Notebook, we'll discuss what Features and Feature Stores are, how they are used in Machine Learning, and various Feature Engineering techniques.

## **Recap**
##### Here is a quick review of the steps we took before Data Preprocessing:
###### 1. **Data Sources**: We identified relevant data sources.
###### 2. **Data Ingestion**: We got the data from the data sources identified.
###### 3. **Data Storage**: We found a temporary storage repository for the data we ingested.
###### 4. **Data Wrangling**: We cleaned the data to make it useable for analysis.
###### 5. **Exploratory Data Analysis (EDA)**: We identified outliers, relationships between variables, and any missing values.
###### 6. **Data Preprocessing**: We formatted the data in a way where the machine can make sense of the data.

### **1. What is Feature Engineering?**

##### ***Feature Engineering*** is the process of selecting, manipulating, and transforming raw data into features that can be used in supervised learning. 
###### (https://towardsdatascience.com/what-is-feature-engineering-importance-tools-and-techniques-for-machine-learning-2080b0269f10)

##### When collecting data at the beginning of the Machine Learning Lifecycle, you'll likely have more than 100 Features to choose from. Although the majority of these may be helpful when predicting or classifying an observation, there will be quite a few that may create "noise" during the training process.

### **2. What is a Feature Store?**

##### **Watch the Following Video for a Short 2-Minute Summary:** https://www.youtube.com/watch?v=l6xfFYZAyns

##### A ***feature store*** for machine learning is a platform that manages and provides access to both historical and live feature data and also provides support for creating point-in-time correct datasets from the historical feature data.

###### (https://www.featurestore.org/what-is-a-feature-store)

##### It is a central location where you can either create or update groups of features created from multiple different data sources, or create and update new datasets from those feature groups for training models or for use in applications that do not want to compute the features but just retrieve them when it needs them to make predictions.

### **3. Feature Selection Techniques**
##### In practice, Feature Selection is traditionally performed before Data Pre-processing.
###### The following information was extracted from the following video: https://www.youtube.com/watch?v=7tW29jBceRw
##### **Why Do we Perform Feature Selection?**
##### Simply put, although modern computers are fast, they can quickly get bogged down when the concept of Big O Notation comes into play. Thus, Feature Selection provides the following advantages:
- Shorten Model Training Times -> Improve Computational Efficiency
- Reduce Generalization Errors in the Model by Removing Irrelevant Features
- Improves the Predicitve Power of the Model and Reduces the Impact of Overfitting
#### **Intrinsic Methods:**
- Some algorithms have Feature Selection processes embedded. These simply put weights on the Features, allowing for less important features to be disregarded.
- Decision Trees put weights on individual Features when finding the best split methods.
- Random Forests uses an ensemble approach where it selects a random subset of Features to use in the tree and compares the relationship between them. It iterates through several combinations of Features until it gets an understanding of how well correlated each Feature is with all the other Features.
- Regularization Models, such as Neural Networks, do something similar where they place an initial weight on all the features and then use a Loss Function to find the optimal weight for each Feature.
#### **Filter Methods**
- When using an algorithm that doesn't involve an Intrinsic Method, you can use Statistical Tests to identify the best subset of Features to use in your model.
- Correlation Matrices can be used to visualize the relationships between the Target Feature and the Predictor Features.
- When using a library such as SciKit-Learn, some algorithms provide a "Feature Importance" result set which uses a Statistical approach to identifying how much each individual Feature contributed to the outcome.
- With Domain Knowledge, or knowledge applied to a specific field of study (e.g., healthcare, marketing/sales), you can use intuition to remove Features that you're confident won't affect the model whatsoever.
#### **Wrapper Methods**
- ML Engineers sometimes take an iterative approach where they first train the model on all the Features and then remove Features one by one until the best result is achieved.
- On the other hand, ML Engineers also start with a single Feature and then add Featrues one at a time until the best result is achieved.
- This is commonly used in Linear and Logistic Regression techniques where p-values are used to identify Feature Importance.

### **Feature Engineering in Action**
##### In the example below, our objective is to predict the price of a used car using the best set of Features we have available.

In [2]:
import pandas as pd

# preview dataset

training_data = pd.read_parquet('clean_car_listings.parquet')
training_data.head()

,price,model_year,make,model,trim,mileage,exterior_color,interior_color,num_accidents,num_owners,usage_type,city,state
0,13895,2006,BMW,Z4,Roadster 3.0si,114889,White,Unknown,0,5,Personal,Tempe,AZ
1,19888,2008,BMW,M5,Sedan,129195,Blue,Black,0,3,Personal,Tempe,AZ
2,19999,2008,BMW,M6,Coupe,93700,Gray,Black,0,2,Fleet,West Park,FL
3,18995,2009,BMW,Z4,Roadster sDrive30i,95185,Gray,Black,1,5,Fleet,Englewood,CO
4,6500,2010,BMW,X3,xDrive30i AWD,126832,Red,Beige,0,3,Personal,Bountiful,UT


##### **Intrinsic Approach**
###### In this example, we'll go through the MLOps process (let's assume we have already performed Data Wrangling and EDA). With the Intrinsic Approach, we'll make use of a Decision Tree to predict the price feature.

In [24]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import plotly.express as px

# preprocess data
encoder = LabelEncoder()
training_data['make'] = encoder.fit_transform(training_data['make'])
training_data['model'] = encoder.fit_transform(training_data['model'])
training_data['trim'] = encoder.fit_transform(training_data['trim'])
training_data['exterior_color'] = encoder.fit_transform(training_data['exterior_color'])
training_data['interior_color'] = encoder.fit_transform(training_data['interior_color'])
training_data['usage_type'] = encoder.fit_transform(training_data['usage_type'])
training_data['city'] = encoder.fit_transform(training_data['city'])
training_data['state'] = encoder.fit_transform(training_data['state'])

# specify predictor and target features
X = training_data.drop(columns=['price'])
y = training_data['price']

# split dataset into Training and Testing Subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# fit training data to model
used_car_model = DecisionTreeRegressor()
used_car_model.fit(X_train, y_train)

# create DataFrame with feature names and values, visualize feature importance chart
features = pd.DataFrame({'value':used_car_model.feature_importances_}, index=[X.columns])
features.reset_index(inplace=True)
features.rename(columns={'level_0':'feature'}, inplace=True)
px.bar(features.sort_values(by='value'), x='value', y='feature')

##### Using our Decision Tree's built-in Intrinsic Method of Feature Engineering, we can identify that **model_year, model**, and **trim** are the 3 most important Features, whereas **num_accidents, usage_type**, and **num_owners** are the three least important Featrues**

#### **Filter Approach**
##### In this example, we'll do the same thing except use a Linear Regression model. To do this, we'll need to exclude all categorical features. Since Linear Regression is based on p-values we'll use the ***statsmodels*** library.

In [32]:
import statsmodels.api as sm

# read training dataset

training_data = pd.read_parquet('clean_car_listings.parquet')

# filter out categorical features
categorical_feats = ['make', 'model', 'trim', 'exterior_color', 'interior_color', 'usage_type', 'city', 'state']
training_data.drop(columns=categorical_feats, inplace=True)

# view correlation matrix

print(training_data.corr())
print('')
print('')
# fit training data to model

used_car_model = sm.OLS(training_data['price'], training_data.drop(columns=['price']))
result = used_car_model.fit()
print(result.summary())

                  price  model_year   mileage  num_accidents  num_owners
price          1.000000    0.571336 -0.567152      -0.164303   -0.310564
model_year     0.571336    1.000000 -0.741074      -0.193095   -0.507374
mileage       -0.567152   -0.741074  1.000000       0.190236    0.435897
num_accidents -0.164303   -0.193095  0.190236       1.000000    0.131187
num_owners    -0.310564   -0.507374  0.435897       0.131187    1.000000


                                 OLS Regression Results                                
Dep. Variable:                  price   R-squared (uncentered):                9093.017
Model:                            OLS   Adj. R-squared (uncentered):           9093.505
Method:                 Least Squares   F-statistic:                         -1.865e+04
Date:                Tue, 17 Oct 2023   Prob (F-statistic):                        1.00
Time:                        14:13:33   Log-Likelihood:                     -7.8959e+05
No. Observations:               

##### Using a Filter Approach, we could remove the "num_accidents" and "num_owners" Features as they have low correlations with the Target Feature. However, when training a Linear or Logistic Regression Model, p-values should always be used as they are the universal indicator of Statistical Significance.
##### When we train the model and view the summary table, we can see the "num_accidents" and "num_owners" Features have p-values lower than 0.05, meaning they are statistically significant, whereas using the Correlation Matrix we would have likely chosen to remove them.

#### **Wrapper Approach**
##### In this example, we'll use the same dataset with the same target feature. However, this time, we'll explore the Forward Sequential and Backward Sequential Approaches.

In [45]:
import statsmodels.api as sm

# read training dataset

training_data = pd.read_parquet('clean_car_listings.parquet')

# filter out categorical features
categorical_feats = ['make', 'model', 'trim', 'exterior_color', 'interior_color', 'usage_type', 'city', 'state']
training_data.drop(columns=categorical_feats, inplace=True)

# view correlation matrix
print('Correlation Matrix')
print(training_data.corr())
print('')
print('')

'''
Forward Sequential Approach. In this approach, we start with one Feature and add more features until we get the best result.
'''

# fit training data to model with 1 feature.
used_car_model = sm.OLS(training_data['price'], training_data.drop(columns=['mileage', 'num_accidents', 'num_owners', 'price']))
result = used_car_model.fit()
summary = result.summary()
results_as_html = summary.tables[1].as_html()
summary = pd.read_html(results_as_html, header=0, index_col=0)[0]
print('1 Feature Model Summary')
print(summary)
print('')

# fit training data to model with 2 Features.
used_car_model = sm.OLS(training_data['price'], training_data.drop(columns=['num_accidents', 'num_owners', 'price']))
result = used_car_model.fit()
summary = result.summary()
results_as_html = summary.tables[1].as_html()
summary = pd.read_html(results_as_html, header=0, index_col=0)[0]
print('2 Feature Model Summary')
print(summary)
print('')

# fit training data to model with 3 Features
used_car_model = sm.OLS(training_data['price'], training_data.drop(columns=['num_owners', 'price']))
result = used_car_model.fit()
summary = result.summary()
results_as_html = summary.tables[1].as_html()
summary = pd.read_html(results_as_html, header=0, index_col=0)[0]
print('3 Feature Model Summary')
print(summary)
print('')

# fit training data to model with 4 Features
used_car_model = sm.OLS(training_data['price'], training_data.drop(columns=['price']))
result = used_car_model.fit()
summary = result.summary()
results_as_html = summary.tables[1].as_html()
summary = pd.read_html(results_as_html, header=0, index_col=0)[0]
print('4 Feature Model Summary')
print(summary)
print('')

'''
Backward Sequential Approach. In this approach, we start with all features and remove features as necessary.
'''
# fit training data to model with All Features
used_car_model = sm.OLS(training_data['price'], training_data.drop(columns=['price']))
result = used_car_model.fit()
summary = result.summary()
results_as_html = summary.tables[1].as_html()
summary = pd.read_html(results_as_html, header=0, index_col=0)[0]
print('4 Feature Model Summary')
print(summary)

Correlation Matrix
                  price  model_year   mileage  num_accidents  num_owners
price          1.000000    0.571336 -0.567152      -0.164303   -0.310564
model_year     0.571336    1.000000 -0.741074      -0.193095   -0.507374
mileage       -0.567152   -0.741074  1.000000       0.190236    0.435897
num_accidents -0.164303   -0.193095  0.190236       1.000000    0.131187
num_owners    -0.310564   -0.507374  0.435897       0.131187    1.000000


1 Feature Model Summary
               coef  std err        t  P>|t|  [0.025  0.975]
model_year  13.9712    0.021  659.913    0.0   13.93  14.013

2 Feature Model Summary
               coef  std err        t  P>|t|  [0.025  0.975]
model_year  19.0511    0.032  591.137    0.0  18.988  19.114
mileage     -0.1950    0.001 -187.503    0.0  -0.197  -0.193

3 Feature Model Summary
                    coef  std err        t  P>|t|    [0.025   0.975]
model_year       19.1247    0.032  590.610    0.0    19.061   19.188
mileage          -0.1912